# Data refinement

With this Jupyter Notebook i can do diferent operations in the original images i have to resize, move or translate for my Neural Network.

I have a lot of videos and i m going to take frames from them. I will refine that frames to have the images for our Keras Neural Network

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import glob
import cv2
import os,shutil
import random
import time

def printAndRemove(msg):
    print(msg, end="\r")
    time.sleep(0)

# Extract images from videos

With this code i will take the images from my videos:

In [2]:
#size = 1920, 1080
#size = 960, 540
#size = 480, 270
size = 240, 135
#size = 96, 54

path = ""
saveIn = "data"

if os.path.isdir(path + saveIn):
    shutil.rmtree(path + saveIn)
os.makedirs(path + saveIn)
os.makedirs(path + saveIn + "/train")
os.makedirs(path + saveIn + "/validation")

i=0

imgNumber = 0
trainingPerNode = []
numberImagesInThisNode = 0
for video in glob.glob(path + "Nodes/*.mp4"):
    i=i+1
    newDir = video.split('.')[0].split('_')[0].split('\\')
    newDir = newDir[len(newDir)-1]
    if not os.path.exists(path + saveIn + "/train/" + newDir):
        numberImagesInThisNode = 0
        os.makedirs(path + saveIn + "/train/" + newDir)
    if not os.path.exists(path + saveIn + "/validation/" + newDir):
        os.makedirs(path + saveIn + "/validation/" + newDir)
    
    cap = cv2.VideoCapture(video)
    frameWidth = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frameHeight = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    currentframe = 0
    
    while(cap.isOpened()):
        ret, frame = cap.read()
        if ret:
            printAndRemove("Working on image nº" + str(imgNumber))
            
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame = Image.fromarray(frame)
            if(i != 8):
                frame = frame.rotate(180)
            frame.thumbnail(size, Image.ANTIALIAS)
            frame.save(path + saveIn + "/train/" + newDir + "/img" + str(imgNumber) + ".jpg", "JPEG")
            
            numberImagesInThisNode += 1
            imgNumber += 1
        else:
            break
    trainingPerNode.append(numberImagesInThisNode)
cap.release()
cv2.destroyAllWindows()

validationPerNode = []
node=0
for directory in glob.glob(path + saveIn + "/train/*"):
    directory = directory.split('\\')[1]
    validationImages = 0
    while validationImages < trainingPerNode[node]/10:
        for img in glob.glob(path + saveIn + "/train/" + directory + "/*.jpg"):
            if(validationImages < trainingPerNode[node]/10):
                r = random.randint(0, 100)
                if(r<15):
                    shutil.move(img, path + saveIn + "/validation/" + directory)
                    validationImages += 1
    node += 1
    validationPerNode.append(validationImages)


print("")
print("Done!")

Working on image nº22404
Done!


# Count images

With this code i will count all the data i have

In [36]:
path = ""

def countImages(directory):
    count=0
    for img in glob.glob(directory + "/*.jpg"):
        count = count + 1
    return count

training = []
print("Training\n")
for directory in glob.glob(path + "data/train/*"):
    numImages = countImages(directory)
    nodeName = directory.split('\\')[1]
    print(nodeName + "")
    print("\tTotal images in "+ str(nodeName) +": " +  str(numImages))
    training.append(numImages)

print("\nValidation\n")
for directory in glob.glob(path + "data/validation/*"):
    numImages = countImages(directory)
    nodeName = directory.split('\\')[1]
    print(nodeName + "")
    print("\tTotal images in "+ str(nodeName) +": " +  str(numImages))

Training

Nodo1
	Total images in Nodo1: 5169
Nodo2
	Total images in Nodo2: 3234
Nodo3
	Total images in Nodo3: 2369
Nodo4
	Total images in Nodo4: 4835
Nodo5
	Total images in Nodo5: 3377
Nodo6
	Total images in Nodo6: 1480

Validation

Nodo1
	Total images in Nodo1: 121
Nodo2
	Total images in Nodo2: 228
Nodo3
	Total images in Nodo3: 316
Nodo4
	Total images in Nodo4: 340
Nodo5
	Total images in Nodo5: 426
Nodo6
	Total images in Nodo6: 510


# Disperse images

With this code i will disperse all the images i have. With this i expect to dont have our images in the training in a sequential order and improve the training

In [37]:
import random

def disperse(directory):
    # We create the temporal file
    if os.path.isdir(directory + "temp"):
        shutil.rmtree(directory + "temp")
    os.makedirs(directory + "temp")
    # We count the images
    numberOfImagesInDirectory = countImages(directory)
    # We change the name of all the images in the file
    for image in glob.glob(directory + "/*.jpg"):
        nameIsUsed = True
        newName = ""
        # No collision with other images names
        while(nameIsUsed):
            newName = "img" + str(numberOfImagesInDirectory + random.randint(0, 10000))
            if(os.path.isfile(directory + "temp/" + newName + ".jpg")):
                nameIsUsed = True
            else:
                nameIsUsed = False
        # We rename the file
        os.rename(image, directory + "temp/" + newName + ".jpg")
    for image in glob.glob(directory + "temp/*.jpg"):
        temporal = image.split("temp")
        shutil.move(image,temporal[0]+temporal[1])
    shutil.rmtree(directory + "temp")

for directory in glob.glob(path + "data/train/*"):
    disperse(directory)

training = []
print("Training\n")
for directory in glob.glob(path + "data/train/*"):
    numImages = countImages(directory)
    nodeName = directory.split('\\')[1]
    print(nodeName + "")
    print("\tTotal images in "+ str(nodeName) +": " +  str(numImages))
    training.append(numImages)

data/train\Nodo1temp\img10000.jpg
data/train\Nodo1\img10000.jpg
data/train\Nodo1temp\img10002.jpg
data/train\Nodo1\img10002.jpg
data/train\Nodo1temp\img10003.jpg
data/train\Nodo1\img10003.jpg
data/train\Nodo1temp\img10004.jpg
data/train\Nodo1\img10004.jpg
data/train\Nodo1temp\img10005.jpg
data/train\Nodo1\img10005.jpg
data/train\Nodo1temp\img10007.jpg
data/train\Nodo1\img10007.jpg
data/train\Nodo1temp\img10008.jpg
data/train\Nodo1\img10008.jpg
data/train\Nodo1temp\img10009.jpg
data/train\Nodo1\img10009.jpg
data/train\Nodo1temp\img10010.jpg
data/train\Nodo1\img10010.jpg
data/train\Nodo1temp\img10011.jpg
data/train\Nodo1\img10011.jpg
data/train\Nodo1temp\img10013.jpg
data/train\Nodo1\img10013.jpg
data/train\Nodo1temp\img10015.jpg
data/train\Nodo1\img10015.jpg
data/train\Nodo1temp\img10019.jpg
data/train\Nodo1\img10019.jpg
data/train\Nodo1temp\img10020.jpg
data/train\Nodo1\img10020.jpg
data/train\Nodo1temp\img10021.jpg
data/train\Nodo1\img10021.jpg
data/train\Nodo1temp\img10023.jpg
data/t

data/train\Nodo1\img10399.jpg
data/train\Nodo1temp\img10400.jpg
data/train\Nodo1\img10400.jpg
data/train\Nodo1temp\img10401.jpg
data/train\Nodo1\img10401.jpg
data/train\Nodo1temp\img10403.jpg
data/train\Nodo1\img10403.jpg
data/train\Nodo1temp\img10405.jpg
data/train\Nodo1\img10405.jpg
data/train\Nodo1temp\img10406.jpg
data/train\Nodo1\img10406.jpg
data/train\Nodo1temp\img10408.jpg
data/train\Nodo1\img10408.jpg
data/train\Nodo1temp\img10409.jpg
data/train\Nodo1\img10409.jpg
data/train\Nodo1temp\img10410.jpg
data/train\Nodo1\img10410.jpg
data/train\Nodo1temp\img10412.jpg
data/train\Nodo1\img10412.jpg
data/train\Nodo1temp\img10418.jpg
data/train\Nodo1\img10418.jpg
data/train\Nodo1temp\img10420.jpg
data/train\Nodo1\img10420.jpg
data/train\Nodo1temp\img10421.jpg
data/train\Nodo1\img10421.jpg
data/train\Nodo1temp\img10423.jpg
data/train\Nodo1\img10423.jpg
data/train\Nodo1temp\img10424.jpg
data/train\Nodo1\img10424.jpg
data/train\Nodo1temp\img10425.jpg
data/train\Nodo1\img10425.jpg
data/train

data/train\Nodo1temp\img10738.jpg
data/train\Nodo1\img10738.jpg
data/train\Nodo1temp\img10740.jpg
data/train\Nodo1\img10740.jpg
data/train\Nodo1temp\img10741.jpg
data/train\Nodo1\img10741.jpg
data/train\Nodo1temp\img10742.jpg
data/train\Nodo1\img10742.jpg
data/train\Nodo1temp\img10743.jpg
data/train\Nodo1\img10743.jpg
data/train\Nodo1temp\img10744.jpg
data/train\Nodo1\img10744.jpg
data/train\Nodo1temp\img10746.jpg
data/train\Nodo1\img10746.jpg
data/train\Nodo1temp\img10748.jpg
data/train\Nodo1\img10748.jpg
data/train\Nodo1temp\img10749.jpg
data/train\Nodo1\img10749.jpg
data/train\Nodo1temp\img10755.jpg
data/train\Nodo1\img10755.jpg
data/train\Nodo1temp\img10757.jpg
data/train\Nodo1\img10757.jpg
data/train\Nodo1temp\img10758.jpg
data/train\Nodo1\img10758.jpg
data/train\Nodo1temp\img10760.jpg
data/train\Nodo1\img10760.jpg
data/train\Nodo1temp\img10762.jpg
data/train\Nodo1\img10762.jpg
data/train\Nodo1temp\img10763.jpg
data/train\Nodo1\img10763.jpg
data/train\Nodo1temp\img10764.jpg
data/t

data/train\Nodo1temp\img11095.jpg
data/train\Nodo1\img11095.jpg
data/train\Nodo1temp\img11096.jpg
data/train\Nodo1\img11096.jpg
data/train\Nodo1temp\img11099.jpg
data/train\Nodo1\img11099.jpg
data/train\Nodo1temp\img11104.jpg
data/train\Nodo1\img11104.jpg
data/train\Nodo1temp\img11105.jpg
data/train\Nodo1\img11105.jpg
data/train\Nodo1temp\img11108.jpg
data/train\Nodo1\img11108.jpg
data/train\Nodo1temp\img11110.jpg
data/train\Nodo1\img11110.jpg
data/train\Nodo1temp\img11111.jpg
data/train\Nodo1\img11111.jpg
data/train\Nodo1temp\img11112.jpg
data/train\Nodo1\img11112.jpg
data/train\Nodo1temp\img11113.jpg
data/train\Nodo1\img11113.jpg
data/train\Nodo1temp\img11114.jpg
data/train\Nodo1\img11114.jpg
data/train\Nodo1temp\img11115.jpg
data/train\Nodo1\img11115.jpg
data/train\Nodo1temp\img11116.jpg
data/train\Nodo1\img11116.jpg
data/train\Nodo1temp\img11117.jpg
data/train\Nodo1\img11117.jpg
data/train\Nodo1temp\img11118.jpg
data/train\Nodo1\img11118.jpg
data/train\Nodo1temp\img11121.jpg
data/t

data/train\Nodo1temp\img11413.jpg
data/train\Nodo1\img11413.jpg
data/train\Nodo1temp\img11415.jpg
data/train\Nodo1\img11415.jpg
data/train\Nodo1temp\img11417.jpg
data/train\Nodo1\img11417.jpg
data/train\Nodo1temp\img11418.jpg
data/train\Nodo1\img11418.jpg
data/train\Nodo1temp\img11420.jpg
data/train\Nodo1\img11420.jpg
data/train\Nodo1temp\img11421.jpg
data/train\Nodo1\img11421.jpg
data/train\Nodo1temp\img11424.jpg
data/train\Nodo1\img11424.jpg
data/train\Nodo1temp\img11425.jpg
data/train\Nodo1\img11425.jpg
data/train\Nodo1temp\img11426.jpg
data/train\Nodo1\img11426.jpg
data/train\Nodo1temp\img11430.jpg
data/train\Nodo1\img11430.jpg
data/train\Nodo1temp\img11432.jpg
data/train\Nodo1\img11432.jpg
data/train\Nodo1temp\img11437.jpg
data/train\Nodo1\img11437.jpg
data/train\Nodo1temp\img11443.jpg
data/train\Nodo1\img11443.jpg
data/train\Nodo1temp\img11444.jpg
data/train\Nodo1\img11444.jpg
data/train\Nodo1temp\img11448.jpg
data/train\Nodo1\img11448.jpg
data/train\Nodo1temp\img11450.jpg
data/t

...

data/train\Nodo6temp\img3849.jpg
data/train\Nodo6\img3849.jpg
data/train\Nodo6temp\img3851.jpg
data/train\Nodo6\img3851.jpg
data/train\Nodo6temp\img3853.jpg
data/train\Nodo6\img3853.jpg
data/train\Nodo6temp\img3854.jpg
data/train\Nodo6\img3854.jpg
data/train\Nodo6temp\img3858.jpg
data/train\Nodo6\img3858.jpg
data/train\Nodo6temp\img3868.jpg
data/train\Nodo6\img3868.jpg
data/train\Nodo6temp\img3872.jpg
data/train\Nodo6\img3872.jpg
data/train\Nodo6temp\img3873.jpg
data/train\Nodo6\img3873.jpg
data/train\Nodo6temp\img3880.jpg
data/train\Nodo6\img3880.jpg
data/train\Nodo6temp\img3883.jpg
data/train\Nodo6\img3883.jpg
data/train\Nodo6temp\img3884.jpg
data/train\Nodo6\img3884.jpg
data/train\Nodo6temp\img3887.jpg
data/train\Nodo6\img3887.jpg
data/train\Nodo6temp\img3889.jpg
data/train\Nodo6\img3889.jpg
data/train\Nodo6temp\img3900.jpg
data/train\Nodo6\img3900.jpg
data/train\Nodo6temp\img3902.jpg
data/train\Nodo6\img3902.jpg
data/train\Nodo6temp\img3903.jpg
data/train\Nodo6\img3903.jpg
data/tra

data/train\Nodo6temp\img5092.jpg
data/train\Nodo6\img5092.jpg
data/train\Nodo6temp\img5096.jpg
data/train\Nodo6\img5096.jpg
data/train\Nodo6temp\img5104.jpg
data/train\Nodo6\img5104.jpg
data/train\Nodo6temp\img5108.jpg
data/train\Nodo6\img5108.jpg
data/train\Nodo6temp\img5114.jpg
data/train\Nodo6\img5114.jpg
data/train\Nodo6temp\img5120.jpg
data/train\Nodo6\img5120.jpg
data/train\Nodo6temp\img5121.jpg
data/train\Nodo6\img5121.jpg
data/train\Nodo6temp\img5122.jpg
data/train\Nodo6\img5122.jpg
data/train\Nodo6temp\img5137.jpg
data/train\Nodo6\img5137.jpg
data/train\Nodo6temp\img5144.jpg
data/train\Nodo6\img5144.jpg
data/train\Nodo6temp\img5149.jpg
data/train\Nodo6\img5149.jpg
data/train\Nodo6temp\img5150.jpg
data/train\Nodo6\img5150.jpg
data/train\Nodo6temp\img5161.jpg
data/train\Nodo6\img5161.jpg
data/train\Nodo6temp\img5170.jpg
data/train\Nodo6\img5170.jpg
data/train\Nodo6temp\img5178.jpg
data/train\Nodo6\img5178.jpg
data/train\Nodo6temp\img5204.jpg
data/train\Nodo6\img5204.jpg
data/tra

data/train\Nodo6\img6584.jpg
data/train\Nodo6temp\img6592.jpg
data/train\Nodo6\img6592.jpg
data/train\Nodo6temp\img6596.jpg
data/train\Nodo6\img6596.jpg
data/train\Nodo6temp\img6602.jpg
data/train\Nodo6\img6602.jpg
data/train\Nodo6temp\img6604.jpg
data/train\Nodo6\img6604.jpg
data/train\Nodo6temp\img6605.jpg
data/train\Nodo6\img6605.jpg
data/train\Nodo6temp\img6606.jpg
data/train\Nodo6\img6606.jpg
data/train\Nodo6temp\img6609.jpg
data/train\Nodo6\img6609.jpg
data/train\Nodo6temp\img6611.jpg
data/train\Nodo6\img6611.jpg
data/train\Nodo6temp\img6631.jpg
data/train\Nodo6\img6631.jpg
data/train\Nodo6temp\img6639.jpg
data/train\Nodo6\img6639.jpg
data/train\Nodo6temp\img6640.jpg
data/train\Nodo6\img6640.jpg
data/train\Nodo6temp\img6649.jpg
data/train\Nodo6\img6649.jpg
data/train\Nodo6temp\img6650.jpg
data/train\Nodo6\img6650.jpg
data/train\Nodo6temp\img6658.jpg
data/train\Nodo6\img6658.jpg
data/train\Nodo6temp\img6660.jpg
data/train\Nodo6\img6660.jpg
data/train\Nodo6temp\img6661.jpg
data/tra

data/train\Nodo6temp\img7846.jpg
data/train\Nodo6\img7846.jpg
data/train\Nodo6temp\img7847.jpg
data/train\Nodo6\img7847.jpg
data/train\Nodo6temp\img7857.jpg
data/train\Nodo6\img7857.jpg
data/train\Nodo6temp\img7860.jpg
data/train\Nodo6\img7860.jpg
data/train\Nodo6temp\img7866.jpg
data/train\Nodo6\img7866.jpg
data/train\Nodo6temp\img7869.jpg
data/train\Nodo6\img7869.jpg
data/train\Nodo6temp\img7880.jpg
data/train\Nodo6\img7880.jpg
data/train\Nodo6temp\img7885.jpg
data/train\Nodo6\img7885.jpg
data/train\Nodo6temp\img7893.jpg
data/train\Nodo6\img7893.jpg
data/train\Nodo6temp\img7899.jpg
data/train\Nodo6\img7899.jpg
data/train\Nodo6temp\img7901.jpg
data/train\Nodo6\img7901.jpg
data/train\Nodo6temp\img7903.jpg
data/train\Nodo6\img7903.jpg
data/train\Nodo6temp\img7910.jpg
data/train\Nodo6\img7910.jpg
data/train\Nodo6temp\img7917.jpg
data/train\Nodo6\img7917.jpg
data/train\Nodo6temp\img7919.jpg
data/train\Nodo6\img7919.jpg
data/train\Nodo6temp\img7921.jpg
data/train\Nodo6\img7921.jpg
data/tra

data/train\Nodo6temp\img9152.jpg
data/train\Nodo6\img9152.jpg
data/train\Nodo6temp\img9159.jpg
data/train\Nodo6\img9159.jpg
data/train\Nodo6temp\img9165.jpg
data/train\Nodo6\img9165.jpg
data/train\Nodo6temp\img9168.jpg
data/train\Nodo6\img9168.jpg
data/train\Nodo6temp\img9169.jpg
data/train\Nodo6\img9169.jpg
data/train\Nodo6temp\img9170.jpg
data/train\Nodo6\img9170.jpg
data/train\Nodo6temp\img9172.jpg
data/train\Nodo6\img9172.jpg
data/train\Nodo6temp\img9175.jpg
data/train\Nodo6\img9175.jpg
data/train\Nodo6temp\img9176.jpg
data/train\Nodo6\img9176.jpg
data/train\Nodo6temp\img9186.jpg
data/train\Nodo6\img9186.jpg
data/train\Nodo6temp\img9203.jpg
data/train\Nodo6\img9203.jpg
data/train\Nodo6temp\img9210.jpg
data/train\Nodo6\img9210.jpg
data/train\Nodo6temp\img9215.jpg
data/train\Nodo6\img9215.jpg
data/train\Nodo6temp\img9220.jpg
data/train\Nodo6\img9220.jpg
data/train\Nodo6temp\img9227.jpg
data/train\Nodo6\img9227.jpg
data/train\Nodo6temp\img9233.jpg
data/train\Nodo6\img9233.jpg
data/tra